In [1]:
# -*- coding:utf8-*-
import pandas as pd
import numpy as np

# 一个cell做多个输出
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all" 

# 设定df显示的全部所有列
pd.set_option('display.max_columns', None)
# 设定df全部显示的行数
pd.set_option('display.max_rows', 10)

## 一、读写文本格式的数据 
> 还是多熟悉官网的API文档，同时注意版本间的不同；

pandas 自带许多顶级函数read_xxx系列，可以读取多种格式的数据文件，该部分函数的参数有类似的参数，其参数可以分为以下几个大类： 
- 索引：将一个或多个列当做返回的DataFrame处理，以及是否从文件、用户获取列名。
- 类型推断和数据转换：包括用户定义值的转换、和自定义的缺失值标记列表等
- 日期解析：包括组合功能，比如将分散在多个列中的日期时间信息组合成结果中的单个列（该部分相当重要，尤其涉及到层次化日期索引）。
- 迭代：支持大文件进行逐块迭代
- 不规则数据问题：跳过一些行、页脚、注释或其他一些不重要的东西

read_csv/table()重要参数:
- parse_dates: False/True 开关参数，尝试将数据解析为日期，需指出被解析的列
- date_parser: 用于解析日期的函数
- encoding： 指定解析字符集

In [2]:
# 原始数据
!cat ./examples/ex1.csv

# 1 使用read_csv进行读取， 默认分隔符为 ,
df = pd.read_csv('./examples/ex1.csv')
df.head()

# 2 使用read_table读取，但需手动指定分隔符
df = pd.read_table('./examples/ex1.csv', sep=',')
df.head()

# 3 可以指定columns name和index 
names = ['a','b','c','d','message']
df = pd.read_csv('./examples/ex2.csv',names=names, index_col='message')
df.head()

# 4 在读取时设置层次化索引
df = pd.read_csv('./examples/csv_mindex.csv', index_col=['key1','key2'])
df.head()

# 5 指定不同的分隔符
df = pd.read_csv('./examples/ex3.txt', sep='\s+')  # '\s+' 若干空白字符 与 '\t' 不同
df.head()

# 其余相关属性参考官网api
# help(pd.read_csv)

a,b,c,d,message
1,2,3,4,hello
5,6,7,8,world
9,10,11,12,foo


,a,b,c,d,message
0,1,2,3,4,hello
1,5,6,7,8,world
2,9,10,11,12,foo


,a,b,c,d,message
0,1,2,3,4,hello
1,5,6,7,8,world
2,9,10,11,12,foo


,a,b,c,d
message,,,,
hello,1,2,3,4
world,5,6,7,8
foo,9,10,11,12


value1  value2
key1 key2                
one  a          1       2
     b          3       4
     c          5       6
     d          7       8
two  a          9      10

,A,B,C
aaa,-0.264438,-1.026059,-0.619500
bbb,0.927272,0.302904,-0.032399
ccc,-0.264273,-0.386314,-0.217601
ddd,-0.871858,-0.348382,1.100491


### 逐块读取文本文件 
> 在处理很大的文件时，只想读取文件的一小部分或逐块对文件进行迭代;比自定义逐块读取效率更高；
- 如果只读取几行数据（避免加载整个数据文件）， 只用nrows参数指定
- 要逐块读取文件，指定chunksize参数，一次只读取指定行数的数据，然后内部循环组合(内存中只保存读取逻辑，在需要时在循环读取文件)

In [3]:
# 如果只想读取几行（避免读取整个文件），通过nrows进行指定即可
df = pd.read_csv('./examples/ex6.csv', nrows=3)
df.head()

chunker = pd.read_csv('./examples/ex6.csv', chunksize=1000)
tot = pd.Series([])
for piece in chunker:
    tot = tot.add(piece['key'].value_counts(), fill_value=0)

tot = tot.sort_values(ascending=False)
tot[:10]

,one,two,three,four,key
0,0.467976,-0.038649,-0.295344,-1.824726,L
1,-0.358893,1.404453,0.704965,-0.200638,B
2,-0.501840,0.659254,-0.421691,-0.057688,G


E    368.0
X    364.0
L    346.0
O    343.0
Q    340.0
M    338.0
J    337.0
F    335.0
K    334.0
H    330.0
dtype: float64

### 将数据写出到文本格式
> 与read系列函数类似，只是将数据写到文本保存在指定路径下； 

注意：to_xxx方法与read_xxx方法不同，该方法是df/series对象的方法， 不是顶级方法

In [4]:
df = pd.read_csv('./examples/ex5.csv')
df.head()

df.to_csv('./examples/ex5_out.csv') # 如果不指定，将包含index的数据全部写到文件中

!cat examples/ex5_out.csv
# help(pd.DataFrame.to_csv)

,something,a,b,c,d,message
0,one,1,2,3.0,4,NaN
1,two,5,6,NaN,8,world
2,three,9,10,11.0,12,foo


,something,a,b,c,d,message
0,one,1,2,3.0,4,
1,two,5,6,,8,world
2,three,9,10,11.0,12,foo


### 处理分隔符格式
- 单分隔符文件，使用其他包处理文件中内容上的字符：使用csv模块读取文件，将原始数据中的双引号退去

In [5]:
!cat ./examples/ex7.csv

import csv
f = open('examples/ex7.csv')
reader = csv.reader(f)

for line in reader:
    print(line)

"a","b","c"
"1","2","3"
"1","2","3"
['a', 'b', 'c']
['1', '2', '3']
['1', '2', '3']


In [6]:
with open('examples/ex7.csv') as f:
    lines = list(csv.reader(f))

header, values = lines[0], lines[1:]
data_dict = {h: v for h, v in zip(header, zip(*values))}
data_dict

{'a': ('1', '1'), 'b': ('2', '2'), 'c': ('3', '3')}

In [7]:
# 使用csv读取自定义csv文件
class my_dialect(csv.Dialect):
    lineterminator = '\n' # 行结束符
    delimiter = ';' # 分割符
    quotechar = '"' # 引用符
    quoting = csv.QUOTE_MINIMAL # 引用约定

f = open('./examples/ex7.csv')    
reader = csv.reader(f, dialect=my_dialect())

In [8]:
# 接受一个已打开且可写的文件对象以及跟csv.reader相同的那些语支和格式化选项
with open('./examples/mydata.csv', 'w') as f:
    writer = csv.writer(f, dialect=my_dialect)
    writer.writerow(('one', 'two', 'three'))
    writer.writerow(('1', '2', '3'))
    writer.writerow(('4', '5', '6'))
    writer.writerow(('7', '8', '9'))

df = pd.read_csv('./examples/mydata.csv', sep=';')
df.head()

14

6

6

6

,one,two,three
0,1,2,3
1,4,5,6
2,7,8,9


### JSON数据  
直接使用pd.read_json函数读取即可；若数据需要单纯处理，使用json模块进行读取和转存

In [9]:
obj = """
{"name": "Wes",
"places_lived": ["United States", "Spain", "Germany"],
"pet": null,
"siblings": [{"name": "Scott", "age": 30, "pets": ["Zeus", "Zuko"]},
{"name": "Katie", "age": 38,
"pets": ["Sixes", "Stache", "Cisco"]}]
}
"""

import json
result = json.loads(obj)
result

asjson = json.dumps(result) # json.dumps则将Python对象转换成JSON格式

{'name': 'Wes',
 'places_lived': ['United States', 'Spain', 'Germany'],
 'pet': None,
 'siblings': [{'name': 'Scott', 'age': 30, 'pets': ['Zeus', 'Zuko']},
  {'name': 'Katie', 'age': 38, 'pets': ['Sixes', 'Stache', 'Cisco']}]}

In [10]:
siblings = pd.DataFrame(result['siblings'], columns=['name', 'age'])
siblings

,name,age
0,Scott,30
1,Katie,38


In [11]:
!cat ./examples/example.json

data = pd.read_json('./examples/example.json')
data

[{"a": 1, "b": 2, "c": 3},
 {"a": 4, "b": 5, "c": 6},
 {"a": 7, "b": 8, "c": 9}]


,a,b,c
0,1,2,3
1,4,5,6
2,7,8,9


### XML和HTML：Web信息收集

In [20]:
# import lxml,requests
tables = pd.read_html('./examples/fdic_failed_bank_list.html')
tables[0].head()

,Bank Name,City,ST,CERT,Acquiring Institution,Closing Date,Updated Date
0,Allied Bank,Mulberry,AR,91,Today's Bank,"September 23, 2016","November 17, 2016"
1,The Woodbury Banking Company,Woodbury,GA,11297,United Bank,"August 19, 2016","November 17, 2016"
2,First CornerStone Bank,King of Prussia,PA,35312,First-Citizens Bank & Trust Company,"May 6, 2016","September 6, 2016"
3,Trust Company Bank,Memphis,TN,9956,The Bank of Fayette County,"April 29, 2016","September 6, 2016"
4,North Milwaukee State Bank,Milwaukee,WI,20364,First-Citizens Bank & Trust Company,"March 11, 2016","June 16, 2016"


### 利用lxml.objectify解析XML

In [21]:
from lxml import objectify
path = 'datasets/mta_perf/Performance_MNR.xml'
parsed = objectify.parse(open(path))
root = parsed.getroot()
root

<Element PERFORMANCE at 0x8d41648>

In [22]:
data = []
skip_fields = ['PARENT_SEQ', 'INDICATOR_SEQ','DESIRED_CHANGE', 'DECIMAL_PLACES']
for elt in root.INDICATOR:
    el_data = {}
    for child in elt.getchildren():
        if child.tag in skip_fields:
            continue
        el_data[child.tag] = child.pyval
    data.append(el_data)

perf = pd.DataFrame(data)
perf.head()

,AGENCY_NAME,CATEGORY,DESCRIPTION,FREQUENCY,INDICATOR_NAME,INDICATOR_UNIT,MONTHLY_ACTUAL,MONTHLY_TARGET,PERIOD_MONTH,PERIOD_YEAR,YTD_ACTUAL,YTD_TARGET
0,Metro-North Railroad,Service Indicators,Percent of commuter trains that arrive at thei...,M,On-Time Performance (West of Hudson),%,96.9,95,1,2008,96.9,95
1,Metro-North Railroad,Service Indicators,Percent of commuter trains that arrive at thei...,M,On-Time Performance (West of Hudson),%,95,95,2,2008,96,95
2,Metro-North Railroad,Service Indicators,Percent of commuter trains that arrive at thei...,M,On-Time Performance (West of Hudson),%,96.9,95,3,2008,96.3,95
3,Metro-North Railroad,Service Indicators,Percent of commuter trains that arrive at thei...,M,On-Time Performance (West of Hudson),%,98.3,95,4,2008,96.8,95
4,Metro-North Railroad,Service Indicators,Percent of commuter trains that arrive at thei...,M,On-Time Performance (West of Hudson),%,95.8,95,5,2008,96.6,95


## 二进制数据
> 需要注意的是版本兼容问题；

主要是使用python内置的pickle进行数据序列化，转存为二级制文件

In [23]:
frame = pd.read_csv('examples/ex1.csv')
frame.head()

frame.to_pickle('examples/frame_pickle')
pd.read_pickle('examples/frame_pickle')

,a,b,c,d,message
0,1,2,3,4,hello
1,5,6,7,8,world
2,9,10,11,12,foo


,a,b,c,d,message
0,1,2,3,4,hello
1,5,6,7,8,world
2,9,10,11,12,foo


### 使用HDF5格式 
> 虽然可以用PyTables或h5py库直接访问HDF5文件，pandas提供了更为高级的接口，可以简化存储Series和DataFrame对象。
HDFStore类可以像字典一样，处理低级的细节 

需要安装第三方模块tables：conda install -n python36 tables

In [25]:
frame = pd.DataFrame({'a': np.random.randn(100)})
store = pd.HDFStore('./examples/mydata.h5')
store['obj1'] = frame
store['obj1_col'] = frame['a']
store

# HDF5文件中的对象可以通过与字典一样的API进行获取
store['obj1'].head()

<class 'pandas.io.pytables.HDFStore'>
File path: ./examples/mydata.h5

,a
0,-1.172285
1,2.226966
2,-1.065733
3,0.352905
4,-0.583097


### 读取Excel文件
使用pd.ExcelFile创建对象，在此对象上进行操作；也可以直接对文件进行操作

In [26]:
# 读取
xlsx = pd.ExcelFile('examples/ex1.xlsx')
pd.read_excel(xlsx, 'Sheet1')

pd.read_excel('examples/ex1.xlsx', 'Sheet1')

# 存储
writer = pd.ExcelWriter('examples/ex2.xlsx')
frame.to_excel(writer, 'Sheet1')
writer.save()

frame.to_excel('examples/ex2.xlsx')

,a,b,c,d,message
0,1,2,3,4,hello
1,5,6,7,8,world
2,9,10,11,12,foo


,a,b,c,d,message
0,1,2,3,4,hello
1,5,6,7,8,world
2,9,10,11,12,foo


## Web APIs交互 
需要提供数据的api，也可以直接下载后在进行操作

In [27]:
import requests
url = 'https://api.github.com/repos/pandas-dev/pandas/issues'
resp = requests.get(url)
data = resp.json()

issues = pd.DataFrame(data, columns=['number', 'title', 'labels', 'state'])
issues.head()

,number,title,labels,state
0,25529,fix segfault when running with cython coverage...,[],open
1,25528,DOC: Polishing typos out of doc/source/user_gu...,[],open
2,25527,Data dependent bug in mode aggregation,"[{'id': 307649777, 'node_id': 'MDU6TGFiZWwzMDc...",open
3,25526,DOC: Fixed PeriodArray api ref,"[{'id': 134699, 'node_id': 'MDU6TGFiZWwxMzQ2OT...",open
4,25525,DOC: resolve all GL03 docstring validation errors,"[{'id': 134699, 'node_id': 'MDU6TGFiZWwxMzQ2OT...",open


## 数据库交互
书中的教程参考即可，主要还是不同数据库的配置和几个相关函数(read_sql/read_sql_query/read_sql_table)的使用

```python
import cx_Oracle as orcl
import pymysql as mysql
import os
os.environ['NLS_LANG'] = 'SIMPLIFIED CHINESE_CHINA.UTF8'  # 为oracle设置

# 传统驱动的conn创建
orcl_conn = orcl.connect('user/passwd@ip:1521/orcl')
mysql_conn = mysql.connect(host='ip', port=3306, user='', passwd='', db='',charset="utf8") # 如果数据表中存在中文，必须指定charset参数

# 测试： 如果sql含中文，最好将sql转换为gb18030编码形式
orcl_sql = 'SELECT * FROM TEMP_20131231'
orcl_cursor = orcl_conn.cursor()
orcl_cursor.execute(orcl_sql.decode('utf-8').encode('gb18030'))
mysql_sql = 'select * from temp3'
mysql_cursor = mysql_conn.cursor()
mysql_cursor.execute(mysql_sql.decode('utf-8').encode('gb18030'))

# 使用SQLAlchemy 做连接驱动 不过如果需求没有涉及到ORM模型，不建议使用该模块
from sqlalchemy import create_engine  
conn = create_engine('mysql+pymysql://user:passwd@ip:3306/db?charset=utf8')
```